In [ ]:
# Import required libraries
import os
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString

# Define input parameters 
folder = "./closed_roads"       # Folder containing closed road text files
start_time = 0                  # Start time for rerouting intervals
interval = 3600                   # Interval duration
output_dir = "./rerouted_files"  # Output directory for rerouted XML files
bridge_file = "./bridges.txt"    # File containing bridges to exclude
final_output = "./final_rerouter.xml"  # Final output file

# Function to generate rerouter XML files
def generate_rerouter_files(folder_path, start_time, interval, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    closed_edges_files = sorted([os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".txt")])

    if not closed_edges_files:
        print("No .txt files found in the specified folder.")
        return
    
    for i, closed_edges_file in enumerate(closed_edges_files):
        begin_time = start_time + i * interval
        end_time = begin_time + interval

        with open(closed_edges_file, "r") as file:
            closed_edges = [line.strip() for line in file]

        root = ET.Element("additional")
        for rerouter_id, edge in enumerate(closed_edges):
            rerouter = ET.SubElement(root, "rerouter", {
                "id": f"rerouter_{i}_{rerouter_id}",
                "edges": edge,
                "probability": "1"
            })
            interval_elem = ET.SubElement(rerouter, "interval", {
                "begin": str(begin_time),
                "end": str(end_time)
            })
            ET.SubElement(interval_elem, "closingReroute", {
                "id": edge,
                "disallow": "all"
            })

        rough_string = ET.tostring(root, encoding="utf-8")
        reparsed = parseString(rough_string)
        pretty_xml = reparsed.toprettyxml(indent="    ")

        base_name = os.path.basename(closed_edges_file).replace(".txt", "")
        output_file = os.path.join(output_dir, f"{base_name}_rerouter.add.xml")

        with open(output_file, "w", encoding="utf-8") as file:
            file.write(pretty_xml)

        print(f"Rerouter file saved: {output_file}")

# Function to combine rerouter files
def combine_additional_files(output_dir, output_file):
    combined_root = ET.Element("additional")
    rerouters = []
    added_rerouter_ids = set()

    additional_files = sorted([os.path.join(output_dir, file) for file in os.listdir(output_dir) if file.endswith(".add.xml")])
    
    if not additional_files:
        print("No .add.xml files found in the specified folder.")
        return
    
    for file_path in additional_files:
        tree = ET.parse(file_path)
        root = tree.getroot()

        for rerouter in root.findall("rerouter"):
            interval = rerouter.find("interval")
            begin_time = int(interval.attrib["begin"]) if interval is not None else 0
            rerouter_id = rerouter.attrib["id"]

            if rerouter_id not in added_rerouter_ids:
                rerouters.append((begin_time, rerouter))
                added_rerouter_ids.add(rerouter_id)

    rerouters.sort(key=lambda x: x[0])

    for _, rerouter in rerouters:
        combined_root.append(rerouter)

    rough_string = ET.tostring(combined_root, encoding="utf-8")
    reparsed = parseString(rough_string)
    pretty_xml = reparsed.toprettyxml(indent="    ")

    with open(output_file, "w", encoding="utf-8") as file:
        file.write(pretty_xml)

    print(f"Combined and sorted rerouter file saved to: {output_file}")

# Function to Exclude Bridges from Rerouter
def exclude_edges_from_rerouter(rerouter_file, bridge_file, output_file):
    with open(bridge_file, "r") as file:
        exclude_edges = set(line.strip() for line in file)
    
    tree = ET.parse(rerouter_file)
    root = tree.getroot()
    
    for rerouter in list(root.findall("rerouter")):
        edges = rerouter.get("edges")
        if edges in exclude_edges:
            root.remove(rerouter)
    
    tree.write(output_file, encoding="utf-8", xml_declaration=True)
    print(f"Updated rerouter file saved: {output_file}")

# Function to combine rerouters with the same time intervals
def combine_rerouters(input_file, output_file):
    tree = ET.parse(input_file)
    root = tree.getroot()
    interval_groups = {}

    for rerouter in root.findall("rerouter"):
        interval = rerouter.find("interval")
        begin = interval.attrib["begin"]
        end = interval.attrib["end"]
        key = (begin, end)
        edges = rerouter.attrib["edges"]
        if key not in interval_groups:
            interval_groups[key] = []
        interval_groups[key].append(edges)

    new_root = ET.Element("additional")
    for i, ((begin, end), edges_list) in enumerate(interval_groups.items()):
        combined_rerouter = ET.SubElement(new_root, "rerouter", {
            "id": f"combined_rerouter_{i}",
            "edges": " ".join(edges_list),
            "probability": "1"
        })
        interval_elem = ET.SubElement(combined_rerouter, "interval", {"begin": begin, "end": end})
        for edge in edges_list:
            ET.SubElement(interval_elem, "closingReroute", {"id": edge, "disallow": "all"})

    rough_string = ET.tostring(new_root, encoding="utf-8")
    reparsed = parseString(rough_string)
    pretty_xml = reparsed.toprettyxml(indent="    ")

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(pretty_xml)
    print(f"Final rerouter file saved: {output_file}")

# Execute the Functions Sequentially

# Step 1: Generate rerouter files
generate_rerouter_files(folder, start_time, interval, output_dir)

# Step 2: Combine rerouter files
combined_file = os.path.join(output_dir, "combined_rerouter.xml")
combine_additional_files(output_dir, combined_file)

# Step 3: Exclude bridges from rerouter
excluded_file = os.path.join(output_dir, "rerouter_excluding_bridges.xml")
exclude_edges_from_rerouter(combined_file, bridge_file, excluded_file)

# Step 4: Merge final rerouter
combine_rerouters(excluded_file, final_output)
